In [ ]:
### НУЖНЫЕ ИМПОРТЫ
# !pip install python-dotenv==1.0.1 mistralai

## 1. USER QUERY

In [1]:
# Пока вопрос в виде строки, потом видимо хотим сделать как .py скрипт
user_query = "Как заселить НИП"

## 2. RETRIEVAL
#### 2.1 BUILDING QUERY EMBEDDINGS

#### 2.2 CHANKING DATA

наверное нужно все данные заранее сделать, а не делать это все в онлайне, просто BERT офигеет столько данных через себя прогонять

#### 2.3 SEARCH

## 3. PROMPT BUILDING

## 4. LLM ANSWER

In [2]:
from TerrariaRAG import TerrariaRAG

terraria_rag = TerrariaRAG()
question = "Сколько всего кроватей в террария"
answer = terraria_rag.generate_response(question)
print("\n💬 Вопрос:", question)
print("🧠 Ответ:", answer)
print("\n📚 Источники:")

d:\дз\TerrariaRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing TerrariaRAG components...


d:\дз\TerrariaRAG\src\TerrariaRAG.py:79: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  self.vectorstore = Chroma(persist_directory="./terraria_db", embedding_function=self.embeddings)


TerrariaRAG initialized successfully.

💬 Вопрос: Сколько всего кроватей в террария
🧠 Ответ: В Terraria есть несколько типов кроватей, которые можно использовать для спавна игроков. Вот список всех типов кроватей в игре:

* Кровать из дерева
* Кровать из железных прутьев
* Кровать из мрамора
* Кровать из кристального мрамора
* Кровать из пепельного дерева
* Кровать из Лихзарда
* Кровать из королевского дерева
* Кровать из рубинового дерева
* Кровать из золотого дерева
* Кровать из платинового дерева

Всего в Terraria 10 типов кроватей.

📚 Источники:
